In [ ]:
cd ../src

In [ ]:
import ipynb
from ipynb.fs.full import international_covid_data as cdi
import pandas as pd

In [ ]:
def normalize(x, population):
    return (x/population)*100000

#CONVERT FROM TOTAL CASES TO NEW CASES
df = cdi.get_international_cases()
date_column = df['date']
cases = df.loc[:,df.columns[1:]].fillna(value=0)
new_cases = cases.diff().fillna(value=0)
new_cases = new_cases.astype(int)
new_cases.insert(loc=0, column="Date", value=date_column)
new_cases = new_cases.loc[1:]
population_df = cdi.get_international_population()
population_df = population_df.rename(columns={"countriesAndTerritories": "country"})
new_cases = new_cases.rename(columns={"United States": "United States of America"})

#REPLACE SPACES WITH UNDERSCORE IN COLUMN NAMES IN CASES DATA
new_cases.columns = new_cases.columns.str.replace(' ','_')

#CALCULATE NORMALIZED VALUES
for country in new_cases.columns[1:]:
    population = population_df[population_df['country'] == country]['population']
    if(population.empty): 
        new_cases = new_cases.drop(country, 1)
        continue
    new_cases[country] = new_cases[country].apply(normalize, args=(population,))

new_cases.to_csv('../data/Covid_International/new_cases_normalized.csv', mode='w', index = False, header=True)